# Notebook 00 — Historical Backfill Engine (Polygon → DuckDB)

This notebook builds a **real historical dataset** for the Volatility Alpha Engine (VAE).

We pull 60–180 days of market data from **Polygon**, compute basic volatility stats,
and store everything in **DuckDB**. The other notebooks (01–06) and the RL system
will read from this same DuckDB file.

**Why this matters**

- Makes our project look like real quant research (not toy, single-day data).
- Gives us enough history for EDA, features, regimes, and RL training.
- Shows we can build a reproducible data pipeline.

## 1. Imports and DuckDB connection

**What this cell does**

- Imports Python libraries we need (dates, dataframes, DuckDB).
- Imports our Polygon helper functions from `src/polygon_client.py`.
- Connects to the main DuckDB database file for VAE.

**Why this matters**

- DuckDB is our **single source of truth** for all notebooks.
- Using one DB file keeps the pipeline clean and reproducible.
- We see a real data-engineering pattern, not ad-hoc CSVs.

In [1]:
from pathlib import Path
import sys
from datetime import datetime, timedelta

import duckdb
import pandas as pd
import time
import numpy as np

# --- Make sure we can import from the project root ---
PROJECT_ROOT = Path.cwd().parent  # notebooks/ -> project root
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Now this import will work
from src.polygon_client import get_underlying_bars, compute_realized_vol

# --- Use the SAME DuckDB file as notebooks 1–6 ---
DB_PATH = (PROJECT_ROOT / "data" / "volatility_alpha.duckdb").as_posix()
con = duckdb.connect(DB_PATH)

print("Using DB:", DB_PATH)

Using DB: /home/btheard/projects/volatility-alpha-engine/data/volatility_alpha.duckdb


## 2. Choose tickers and backfill window

**What this cell does**

- Defines the small universe of tickers we care about right now.
- Sets a date window (last ~180 calendar days) to backfill.

**Why this matters**

- ~180 calendar days ≈ 120 trading days, which is enough for:
  - 20-day and 60-day realized volatility
  - Stable features and RL training episodes
- We keep the universe small at first to avoid hitting Polygon rate limits.

In [2]:
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META",
    "NVDA", "TSLA", "AMD", "SPY", "QQQ"]

end_date = datetime.now() # type: ignore
start_date = end_date - timedelta(days=180) # type: ignore

start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

start_date_str, end_date_str, tickers

('2025-06-06',
 '2025-12-03',
 ['AAPL',
  'MSFT',
  'GOOGL',
  'AMZN',
  'META',
  'NVDA',
  'TSLA',
  'AMD',
  'SPY',
  'QQQ'])

## 3. Download daily OHLC bars from Polygon

**What this cell does**

- Loops over each ticker.
- Calls `get_underlying_bars()` to fetch roughly 90 days of daily bars.
- Attaches a `ticker` column and collects all rows in a list.

**Why this matters**

- OHLC (Open, High, Low, Close, Volume) is the core price data
  used in most trading and RL systems.
- This is our **raw market tape** that everything else builds on
  (features, regimes, signals, RL environment).
- We print basic status so we can see which tickers succeeded or failed.

In [3]:
DAYS_HISTORY = 180  # how many calendar days to request from Polygon

all_rows = []
failed = []

for symbol in tickers:  # type: ignore
    print(f"\n🔄 Fetching {symbol}...")
    
    for attempt in range(3):   # up to 3 attempts per symbol
        try:
            bars = get_underlying_bars(symbol, days=DAYS_HISTORY)  # type: ignore

            # If API returns nothing, don't keep retrying
            if bars is None or bars.empty:
                print(f"⚠️ No bars for {symbol} (empty response)")
                break

            # Tag ticker and store
            bars = bars.copy()
            bars["ticker"] = symbol
            all_rows.append(bars)

            print(f"✅ Loaded {len(bars)} bars for {symbol} (attempt {attempt + 1})")
            break  # success → stop retrying this symbol

        except Exception as e:
            print(f"❌ Attempt {attempt + 1} failed for {symbol}: {e}")
            time.sleep(2 * (attempt + 1))  # simple backoff: 2s, 4s, 6s

    else:
        # Only hits if all 3 attempts failed
        print(f"🚫 Giving up on {symbol} after 3 failed attempts")
        failed.append(symbol)

    # Small pause between symbols to be nice to Polygon / your network
    time.sleep(0.5)

print("\n======== Backfill summary ========")
if all_rows:
    print("Successful tickers:", sorted({df_['ticker'].iloc[0] for df_ in all_rows}))
else:
    print("No successful tickers!")

print("Failed tickers:", failed)



🔄 Fetching AAPL...
✅ Loaded 180 bars for AAPL (attempt 1)

🔄 Fetching MSFT...
✅ Loaded 180 bars for MSFT (attempt 1)

🔄 Fetching GOOGL...
✅ Loaded 180 bars for GOOGL (attempt 1)

🔄 Fetching AMZN...
✅ Loaded 180 bars for AMZN (attempt 1)

🔄 Fetching META...
✅ Loaded 180 bars for META (attempt 1)

🔄 Fetching NVDA...
✅ Loaded 180 bars for NVDA (attempt 1)

🔄 Fetching TSLA...
✅ Loaded 180 bars for TSLA (attempt 1)

🔄 Fetching AMD...
✅ Loaded 180 bars for AMD (attempt 1)

🔄 Fetching SPY...
✅ Loaded 180 bars for SPY (attempt 1)

🔄 Fetching QQQ...
✅ Loaded 180 bars for QQQ (attempt 1)

======== Backfill summary ========
Successful tickers: ['AAPL', 'AMD', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'QQQ', 'SPY', 'TSLA']
Failed tickers: []


## 4. Combine and clean the raw bar data

**What this cell does**

- Concatenates all per-ticker DataFrames into a single `df_bars`.
- Converts Polygon `timestamp` to a clean `date` column.
- Shows a preview of the data.

**Why this matters**

- Having all tickers in one DataFrame makes it easy to write to DuckDB.
- A clean `date` column is essential for:
  - Grouping by day
  - Computing returns
  - Aligning features and RL transitions
- This is the **canonical raw table** for downstream notebooks.

In [4]:
# Combine and clean the raw bar data

# What this cell does
# - Concatenate all per-ticker DataFrames into a single df_bars
# - Move Polygon’s time field (index or column) into a proper 'date' column
# - Normalize to calendar dates (YYYY-MM-DD)

if not all_rows:
    raise RuntimeError("No data returned from Polygon. Check API key or rate limits.")

# 1) Combine all tickers' bars into one DataFrame WITHOUT dropping the index
df_bars = pd.concat(all_rows)

print("Backfill bars columns:", list(df_bars.columns))
print("Backfill bars index name:", df_bars.index.name)

cols = df_bars.columns

# 2) Ensure we have a 'date' column from whatever time field Polygon gave us
if df_bars.index.name in ("timestamp", "t"):
    # Time is stored in the index (common with Polygon)
    df_bars = df_bars.reset_index()
    time_col = df_bars.columns[0]          # former index column
    df_bars.rename(columns={time_col: "date"}, inplace=True)
elif "timestamp" in cols:
    df_bars["date"] = df_bars["timestamp"]
elif "t" in cols:
    df_bars["date"] = df_bars["t"]
elif "date" in cols:
    # Already have a date-like column; reuse it
    df_bars["date"] = df_bars["date"]
else:
    raise RuntimeError(
        f"Expected a time column or index in bars, but got columns={list(cols)}, index={df_bars.index.name}"
    )

# 3) Convert to proper datetime, then to calendar date
df_bars["date"] = pd.to_datetime(df_bars["date"], unit="ms", errors="coerce")
if df_bars["date"].isna().all():
    # Fallback if it's already datetime and unit="ms" was wrong
    df_bars["date"] = pd.to_datetime(df_bars["date"], errors="coerce")

if df_bars["date"].isna().all():
    raise RuntimeError("Failed to convert time field to datetime; inspect df_bars.head().")

df_bars["date"] = df_bars["date"].dt.date

df_bars.head()


Backfill bars columns: ['open', 'high', 'low', 'close', 'volume', 'ticker']
Backfill bars index name: timestamp


,date,open,high,low,close,volume,ticker
0,2025-03-18,214.16,215.1500,211.49,212.69,42432426.0,AAPL
1,2025-03-19,214.22,218.7600,213.75,215.24,54385391.0,AAPL
2,2025-03-20,213.99,217.4899,212.22,214.10,48862947.0,AAPL
3,2025-03-21,211.56,218.8400,211.28,218.27,94127768.0,AAPL
4,2025-03-24,221.00,221.4800,218.58,220.73,44299483.0,AAPL


## 5. Compute 20-day and 60-day realized volatility per ticker

**What this cell does**

- For each ticker, sorts the bars by date.
- Uses `compute_realized_vol()` to estimate:
  - 20-day realized volatility (RV20)
  - 60-day realized volatility (RV60)
- Stores one row per ticker in a `df_rv` snapshot table.

**Why this matters**

- Volatility is the heart of this project:
  - It feeds our **Edge Score**.
  - It defines **volatility regimes**.
  - It shapes the **RL state and reward**.
- Having a per-ticker RV snapshot is useful for:
  - Feature sanity checks
  - Comparisons across names
  - UI metrics (e.g., “Avg RV20 across universe”).

In [5]:
# === Build daily realized vol + edge score from 180-day bars ===

# df_bars should already exist from the Polygon backfill:
# columns like: ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']

bars = df_bars.copy()
bars = bars.sort_values(["ticker", "date"])

# 1-day returns
bars["ret"] = bars.groupby("ticker")["close"].pct_change()

# Daily % move (this is what Notebook 1 calls day_pct)
bars["day_pct"] = bars["ret"] * 100.0

ann_factor = np.sqrt(252)

# 20-day realized vol (annualized, in %)
bars["rv_20d"] = (
    bars.groupby("ticker")["ret"]
        .rolling(20)
        .std()
        .reset_index(level=0, drop=True)
        * ann_factor * 100.0
)

# 60-day realized vol (annualized, in %)
bars["rv_60d"] = (
    bars.groupby("ticker")["ret"]
        .rolling(60)
        .std()
        .reset_index(level=0, drop=True)
        * ann_factor * 100.0
)

# Simple “edge” proxy: how big today’s move is vs recent vol
bars["edge_score"] = (bars["day_pct"].abs() / bars["rv_20d"]) * 100.0

# This is the table we’ll persist
df_rv = (
    bars[["date", "ticker", "close", "day_pct", "volume",
          "rv_20d", "rv_60d", "edge_score"]]
      .rename(columns={"date": "run_date", "close": "last_price"})
      .dropna()
)

print("df_rv shape:", df_rv.shape)
df_rv.head()


df_rv shape: (1200, 8)


,run_date,ticker,last_price,day_pct,volume,rv_20d,rv_60d,edge_score
60,2025-06-12,AAPL,199.20,0.211289,43904635.0,20.597902,51.247261,1.025779
61,2025-06-13,AAPL,196.45,-1.380522,51447349.0,20.945016,51.249474,6.591172
62,2025-06-16,AAPL,198.42,1.002800,43020691.0,21.483382,51.291068,4.667793
63,2025-06-17,AAPL,195.64,-1.401068,38856152.0,21.620035,51.185641,6.480417
64,2025-06-18,AAPL,196.58,0.480474,45394689.0,21.672431,51.134823,2.216984


## 6. Write OHLC and volatility tables to DuckDB

**What this cell does**

- Creates `ohlc_bars` table (if it doesn't exist) matching `df_bars` schema.
- Clears any old rows from `ohlc_bars` and inserts the new data.
- Creates `daily_rv` table (if it doesn't exist) matching `df_rv`.
- Clears and refills `daily_rv`.

**Why this matters**

- DuckDB now holds our **canonical raw tables**:
  - `ohlc_bars` = price and volume history
  - `daily_rv` = per-ticker vol snapshot
- All other notebooks (01–06) can reliably read from the same DB.
- This looks like a real quant data warehouse pattern, not ad-hoc CSV dumping.

In [6]:
# === Persist bars & realized-vol tables into DuckDB ===

# 0) Safety: drop old versions so schemas can't conflict
con.execute("DROP TABLE IF EXISTS ohlc_bars;")
con.execute("DROP TABLE IF EXISTS daily_rv;")
con.execute("DROP TABLE IF EXISTS screener_snapshots;")

# 1) OHLC bars (180 days × 10 tickers = 1800 rows)
con.execute("""
CREATE TABLE ohlc_bars AS
SELECT * FROM df_bars;
""")

print("ohlc_bars row count:")
display(con.execute("SELECT COUNT(*) AS n_rows FROM ohlc_bars;").df())

# 2) Daily realized vol + edge (df_rv from the cell above)
con.execute("""
CREATE TABLE daily_rv AS
SELECT * FROM df_rv;
""")

print("daily_rv row count:")
display(con.execute("SELECT COUNT(*) AS n_rows FROM daily_rv;").df())

# 3) Rebuild screener_snapshots using the expected column names
con.execute("""
CREATE TABLE screener_snapshots AS
SELECT
    run_date,
    ticker,
    last_price,
    day_pct,
    volume,
    rv_20d,
    rv_60d,
    edge_score
FROM daily_rv
ORDER BY run_date, ticker;
""")

print("screener_snapshots summary:")
display(con.execute("""
    SELECT
        COUNT(*)               AS n_rows,
        COUNT(DISTINCT ticker) AS n_tickers,
        MIN(run_date)          AS min_date,
        MAX(run_date)          AS max_date
    FROM screener_snapshots;
""").df())


ohlc_bars row count:


,n_rows
0,1800


daily_rv row count:


,n_rows
0,1200


screener_snapshots summary:


,n_rows,n_tickers,min_date,max_date
0,1200,10,2025-06-12,2025-12-02


In [7]:
con.execute("SELECT COUNT(*) AS n_rows FROM daily_rv").df()

,n_rows
0,1200


In [8]:
# Rebuild screener_snapshots from the full daily_rv history

# 1. Drop old table (with just 2 days) if it exists
con.execute("DROP TABLE IF EXISTS screener_snapshots")

# 2. Recreate screener_snapshots from daily_rv
con.execute("""
    CREATE TABLE screener_snapshots AS
    SELECT
        run_date,
        ticker,
        last_price,
        day_pct,
        volume,
        rv_20d,
        rv_60d,
        edge_score
    FROM daily_rv
    ORDER BY run_date, ticker
""")

# 3. Sanity check: should now show 1800 rows and 180-day span
con.sql("""
    SELECT
        COUNT(*)             AS n_rows,
        COUNT(DISTINCT ticker) AS n_tickers,
        MIN(run_date)        AS min_date,
        MAX(run_date)        AS max_date
    FROM screener_snapshots
""").df()

,n_rows,n_tickers,min_date,max_date
0,1200,10,2025-06-12,2025-12-02


## 7. Quick sanity checks

**What this cell does**

- Checks the date range stored in `ohlc_bars`.
- Counts the number of rows per ticker.

**Why this matters**

- Confirms we **actually** have ~60–90 days of data.
- Confirms that all tickers are present and non-empty.
- If something looks off here, we know to fix the data before touching RL.

In [9]:
con.execute("""
SELECT 
    MIN(date) AS min_date,
    MAX(date) AS max_date,
    COUNT(*) AS n_rows
FROM ohlc_bars;
""").fetchdf()

,min_date,max_date,n_rows
0,2025-03-18,2025-12-02,1800


In [10]:
con.execute("""
SELECT 
    ticker,
    COUNT(*) AS n_rows
FROM ohlc_bars
GROUP BY ticker
ORDER BY ticker;
""").fetchdf()

,ticker,n_rows
0,AAPL,180
1,AMD,180
2,AMZN,180
3,GOOGL,180
4,META,180
5,MSFT,180
6,NVDA,180
7,QQQ,180
8,SPY,180
9,TSLA,180


In [11]:
print("Backfill bars columns:", list(df_bars.columns))
print("Backfill bars index name:", df_bars.index.name)
print("Tickers in backfill:", sorted(df_bars["ticker"].unique()))
print("Date range in backfill:", df_bars["date"].min(), "→", df_bars["date"].max())
print("Total rows:", len(df_bars))

Backfill bars columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'ticker']
Backfill bars index name: None
Tickers in backfill: ['AAPL', 'AMD', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'QQQ', 'SPY', 'TSLA']
Date range in backfill: 2025-03-18 → 2025-12-02
Total rows: 1800


In [12]:
con.close()

## 8. Wrap-up: what this notebook proves

**What we did**

- Pulled ~60–90 days of OHLC data from Polygon for a small ticker universe.
- Computed simple realized volatility summaries (RV20, RV60).
- Stored everything in DuckDB tables:
  - `ohlc_bars`
  - `daily_rv`

**Why it matters for VAE and RL**

- All downstream notebooks (01–06) and the RL engine now run on **real history**.
- This enables:
  - More honest EDA and signal analysis
  - Realistic regime detection
  - RL training on multi-day episodes instead of toy examples

**Why it matters**

- Shows you can:
  - Ingest external market data APIs
  - Design a small but solid data model
  - Use DuckDB for analytics
  - Build a reproducible research pipeline

Next step:  
Run notebooks **01 → 06** so the entire VAE pipeline uses this history.
